In [22]:
import pandas as pd
import numpy as np
import itertools
import random

# schema: R(A,B,C)
# query: select agg(C) from R where T.A=2000 and T.B=30;

dom_A = list(range(2000, 2024 + 1))
dom_B = list(range(18, 40 + 1))
doms=[dom_A,dom_B]

print()

def create_db(size): 
    A = random.choices(range(2000,2024),k=size)
    B = random.choices(range(18,41),k=size)
    C = np.random.normal(1000,200,size)
    data = {'A':A,'B':B,'C':C}
    return pd.DataFrame(data)

table=create_db(10000)

true_dom_A = list(table['A'].drop_duplicates().values)
true_dom_B = list(table['B'].drop_duplicates().values)

In [25]:
def run_query_containment(table,low_A,high_A,low_B,high_B,agg_C):
    pres = table.loc[((table['A'] >= low_A) & (table['A'] <= high_A) & (table['B'] >= low_B) & (table['B'] <= high_B))]
    if agg_C=='sum':
        result=pres['C'].sum()
    if agg_C=='avg':
        result=pres['C'].avg()
    if agg_C=='count':
        result=pres['C'].count()
    return result


def run_query(table,cond_A,cond_B,agg_C):
    if '=' in cond_A: 
        table=table[table['A']==int(cond_A.split('=')[1])]
    if '=' in cond_B: 
        table=table[table['B']==int(cond_B.split('=')[1])]
    if '<' in cond_A:
        if '<=' in cond_A:
            table=table[table['A']<=int(cond_A.split('<=')[1])]
        else: 
            table=table[table['A']<int(cond_A.split('<')[1])]
    if '>' in cond_A:
        if '>=' in cond_A:
            table=table[table['A']>=int(cond_A.split('>=')[1])]
        else: 
            table=table[table['A']>int(cond_A.split('>')[1])]
    if '<' in cond_B:
        if '<=' in cond_B:
            table=table[table['B']<=int(cond_B.split('<=')[1])]
        else: 
            table=table[table['B']<int(cond_B.split('<')[1])]
    if '>' in cond_B:
        if '>=' in cond_B:
            table=table[table['B']>=int(cond_B.split('>=')[1])]
        else: 
            table=table[table['B']>int(cond_B.split('>')[1])]
    result = 0
    if agg_C=='sum':
        result=table['C'].sum()
    if agg_C=='avg':
        result=table['C'].avg()
    if agg_C=='count':
        result=table['C'].count()
    return result

#run_query(table,'A>2017','B>=34','sum')

In [26]:
def twoD_space_all(x, y): 
    return [(a, b) for a in x for b in y]

def twoD_space(table, predicate_cols):
    return list(map(tuple, table[predicate_cols].drop_duplicates().to_numpy()))

def twoD_range_containment(table, predicate_cols):
    theta=1
    d = list(map(tuple, table[predicate_cols].drop_duplicates().to_numpy()))
    return [(x[0]-theta,x[1]-theta) for x in d]
    
def twoD_range_containment2(predicate):
    thetas = [i for i in range(10)]
    #return [(predicate[0]-theta, predicate[0]+theta, predicate[1]-theta, predicate[1]+theta) for theta in thetas]
    return [(predicate[0]-theta, predicate[1]-theta) for theta in thetas]
    
def twoD_utility(space, predicate):
    #space = twoD_space(x,y)
    scores = [l1_distance(p1,predicate) for p1 in space]
    # Normalize scores
    mins = min(scores)
    maxs = max(scores)
    return [(s - mins)/(maxs-mins) for s in scores]

def l1_distance(p1,p2):
    return sum(abs(a - b) for a, b in zip(p1, p2))


def twoD_exponential_mechanism(predicate,sensitivity,epsilon,query,space_fn):
    # Create the space and calculate the score for each element in space
    if space_fn==twoD_space_all:
        space=twoD_space_all(doms[0],doms[1])
    if space_fn==twoD_space:
        space = twoD_space(table,['A','B'])
    if space_fn==twoD_range_containment:
        space=twoD_range_containment2(predicate)
        #space=twoD_range_containment(table,['A','B'])
    scores = twoD_utility(space, predicate)
    
    # Calculate the probability for each element, based on its score
    probabilities = [np.exp(epsilon * score / (2 * sensitivity)) for score in scores]
    
    # Normalize the probabilties so they sum to 1
    probabilities = probabilities / np.linalg.norm(probabilities, ord=1)
    
    # Choose an element from R based on the probabilities
    return space[np.random.choice(list(range(0,len(scores))), 1, p=probabilities)[0]]

def accuracy(query,query_p,agg): 
    result = run_query(table,'A='+str(query[0]),'B='+str(query[1]),agg)
    result_p = run_query(table,'A='+str(query_p[0]),'B='+str(query_p[1]),agg)
    return result_p - result


def test_per_query(query,agg,count,space):
    errs = []
    result = run_query(table,'A='+str(query[0]),'B='+str(query[1]),agg)
    for i in range(count): 
        query_p = twoD_exponential_mechanism(query, 1, 1, query, space)
        print('query_p: select %s(C) from R where T.A=%d and T.B=%d;'%(agg,query_p[0],query_p[1]))
        result_p = run_query(table,'A='+str(query_p[0]),'B='+str(query_p[1]),agg)
        error = abs(result_p-result)
        errs+=[error]
    return min(errs), sum(errs)/len(errs), max(errs)


def test_per_query_containment(query,agg,count,space):
    errs = []
    result = run_query(table,'A='+str(query[0]),'B='+str(query[1]),agg)
    for i in range(count): 
        query_p = twoD_exponential_mechanism(query, 1, 1, query, space)
        beta = 5
        print('query_p: select %s(C) from R where T.A>=%d and T.A<=%d and T.B>=%d and T.B<=%d;'%(agg,query_p[0],query[0]+beta,query_p[1],query[1]+beta))
        result_p = run_query_containment(table,query_p[0],query[0]+beta,query_p[1],query_p[1]+beta,agg)
        error = abs(result_p-result)
        errs+=[error]
    return min(errs), sum(errs)/len(errs), max(errs)

def avg(l):
    return sum(l)/len(l)

def test(q_count): 
    agg = 'sum'
    min_err_cont,max_err_cont,avg_err_cont=[],[],[] 
    min_err_res,max_err_res,avg_err_res=[],[],[] 
    for i in range(q_count):
        query = (random.sample(dom_A,1)[0],random.sample(dom_B,1)[0])
        print('query:   select sum(C) from R where T.A=%d and T.B=%d;'%(query[0],query[1]))
        print('correct result: %f'%(run_query(table,'A='+str(query[0]),'B='+str(query[1]),agg)))
        #min_err,avg_err,max_err=test_per_query(query,agg,50,twoD_space_all)
        min_err,avg_err,max_err=test_per_query_containment(query,agg,50,twoD_range_containment)
        min_err_cont+=[min_err]
        avg_err_cont+=[avg_err]
        max_err_cont+=[max_err]
        min_err,avg_err,max_err=test_per_query(query,agg,1,twoD_space)
        min_err_res+=[min_err]
        avg_err_res+=[avg_err]
        max_err_res+=[max_err]
    print('error: twoD_space_containment: min:%f avg: %f max: %f'%(avg(min_err_cont),avg(avg_err_cont),avg(max_err_cont)))
    print('error: twoD_space_restricted: min:%f avg: %f max: %f'%(avg(min_err_res),avg(avg_err_res),avg(max_err_res)))

          
test(1)
          
#  T.A=2000 and T.B=30
#agg = 'sum'
#query = (2017,34)
#print('query:   select sum(C) from R where T.A=%d and T.B=%d;'%(query[0],query[1]))
#print('correct result: %f'%(run_query(table,'A='+str(query[0]),'B='+str(query[1]),agg)))

#min_acc,avg_acc,max_acc = test(query,agg,50,twoD_space_all)
#print('error: twoD_space_all: min:%f avg: %f max: %f'%(min_acc,avg_acc,max_acc))
#min_acc,avg_acc,max_acc = test(query,agg,50,twoD_space)
#print('error: twoD_space:     min:%f avg: %f max: %f'%(min_acc,avg_acc,max_acc))


query:   select sum(C) from R where T.A=2021 and T.B=33;
correct result: 16197.876438
query_p: select sum(C) from R where T.A>=2014 and T.A<=2026 and T.B>=26 and T.B<=38;
query_p: select sum(C) from R where T.A>=2014 and T.A<=2026 and T.B>=26 and T.B<=38;
query_p: select sum(C) from R where T.A>=2012 and T.A<=2026 and T.B>=24 and T.B<=38;
query_p: select sum(C) from R where T.A>=2021 and T.A<=2026 and T.B>=33 and T.B<=38;
query_p: select sum(C) from R where T.A>=2018 and T.A<=2026 and T.B>=30 and T.B<=38;
query_p: select sum(C) from R where T.A>=2015 and T.A<=2026 and T.B>=27 and T.B<=38;
query_p: select sum(C) from R where T.A>=2013 and T.A<=2026 and T.B>=25 and T.B<=38;
query_p: select sum(C) from R where T.A>=2012 and T.A<=2026 and T.B>=24 and T.B<=38;
query_p: select sum(C) from R where T.A>=2012 and T.A<=2026 and T.B>=24 and T.B<=38;
query_p: select sum(C) from R where T.A>=2013 and T.A<=2026 and T.B>=25 and T.B<=38;
query_p: select sum(C) from R where T.A>=2013 and T.A<=2026 and 